In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/mnt


In [ ]:
!pip install --target=$nb_path jdc

In [ ]:
import jdc
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

In [ ]:
class MCAgent():
    def __init__(self, discount, policy):
        raise NotImplementedError

    def agent_start(self, state):
        raise NotImplementedError

    def agent_step(self, reward, state):
        raise NotImplementedError

    def agent_end(self, reward):
        raise NotImplementedError

In [ ]:
%%add_to MCAgent
def __init__(self, discount, policy):

    # Create a random number generator with the provided seed to seed the agent for reproducibility.
    self.rand_generator = np.random.RandomState(0)
    # the discount factor considered
    self.discount = discount
    # policy considered 
    self.policy = policy 
    # array describing number of times a state is visited
    self.n = np.zeros(self.policy.shape[0], )

    # Initialize an array of zeros that will hold the values.
    # Recall that the policy can be represented as a (# States, # Actions) array. With the 
    # assumption that this is the case, we can use the first dimension of the policy to
    # initialize the array for values.
    self.values = np.zeros(self.policy.shape[0], )


In [ ]:
%%add_to MCAgent
def agent_step(self, state):
        
    action = self.rand_generator.choice(range(self.policy.shape[1]), p = self.policy[state]) 
    self.last_state = state
    return action

**Environment class**

Following cells implement Random Walk environment

---



------------
------------
------------

In [ ]:
class RandomWalk():
    def __init__(self, num_states):
        raise NotImplementedError

    def env_start(self, state):
        raise NotImplementedError
    
    def env_step(self, action):
        raise NotImplementedError


In [ ]:
%%add_to RandomWalk
def __init__(self):
    self.states = np.array([0,1,2])
    self. actions = np.array([0,1])
    q_0 = 1/3
    q_1 = 1/2
    q_2 = 1/3
    self.transition_mat_s0 = np.array([[q_0, 1-q_0, 0], [q_0, (1-q_0)/2, (1-q_0)/2]]);
    self.transition_mat_s1 = np.array([[q_0, 1-q_0, 0], [(1-q_1)/2, q_1, (1-q_1)/2], [(1-q_1)/3, q_1, 2*(1-q_1)/3]]);
    self.transition_mat_s2 = np.array([[0, 1-q_2, q_2], [(1-q_2)/3, 2*(1-q_2)/3, q_2]]);

In [ ]:
%%add_to RandomWalk
def env_start(self, state):
    # Arg of env_start is state. This will be given at the start of each episode 
    # to account for exploring starts method. 

    # self.start_state = state
    self.agent_loc = state # start state of agent
    self.reward_mat = np.array([[10, 1],[20, 1],[100, 50]])
    return state

# Note: Initially the reward is zero. We might get confused because of the episode sample which looks like
# S0, A0, R1, S1, A1, R2,... Note that R1 is received "after" you take a step/action from state S0. 
# So when you reach S1, the enviroment gives you the reward of R1. Since, you start with S0, there is no previous state that you 
# have come from. And therfore, when you "reach" S0 or in other words, when you start from S0, you assume that the reward you got
# when you reached S0 is 0 because as mentioned, there is no previous state
# Note that the same idea is carried through. So when you update, you look at both the sides - the state you came from, the state you reached now
# and the reward that you received on reaching the latter state.

In [ ]:
%%add_to RandomWalk
def env_step(self, action):

    # Following lines implement the logic of both the rewards and actions
    reward = self.reward_mat[self.agent_loc][action]

    if self.agent_loc == 0:
        if action == 0:
            self.agent_loc = np.random.choice(self.states, p = self.transition_mat_s0[action - 1])
        if action == 1:
            self.agent_loc = np.random.choice(self.states, p = self.transition_mat_s0[action - 1])
    
    elif self.agent_loc == 1: # if agent in state 1
        if action == 0: # if action left selected
            self.agent_loc = np.random.choice(self.states, p = self.transition_mat_s1[action - 1])
        if action == 1: # if action == 1:
            self.agent_loc = np.random.choice(self.states, p = self.transition_mat_s1[action - 1])
    
    else: # if self.agent_loc == 2:
        if action == 0:
            self.agent_loc = np.random.choice(self.states, p = self.transition_mat_s2[action - 1])
        if action == 1: # if action == 1:
            self.agent_loc = np.random.choice(self.states, p = self.transition_mat_s2[action - 1])
    
    self.reward_state = (reward, self.agent_loc)
    return self.reward_state

In [ ]:
def test_action():
    env = RandomWalk()
    #st = np.random.randint(3)
    env.env_start(2)
    print(env.env_step(1))
    #assert(env.agent_loc == 1)
# for _ in range(50):
#     test_action()


In [ ]:
def test_reward():
    env = RandomWalk()
    env.env_start(1)
    env.env_step(1)
    #assert(env.reward_state[0] == 100)
test_reward()

In [ ]:
# Episode generation
T = 1000 # length of an episode
num_episodes = 100 # number of episodes
states = 3 # number of states
actions = 2 # number of actions
discount = 0.1
epsilon= 0.04
sim = 10000
thres = 0.1
Q = np.zeros((states, actions, sim))
Returns = np.zeros((states, actions, sim))
#N = np.zeros(states, actions)
policy = np.ones((3, 2)) * 0.5
flag = 0
env = RandomWalk()
new_action = 0

for j in range(sim):
    agent = MCAgent(discount, policy)
    s0 = np.random.randint(0, 3)
    sts = [None for _ in range(T+1)]
    acs = [None for _ in range(T+1)]
    rws = np.zeros((3,2))
    state = env.env_start(s0)
    sts[0] = state
    acs[0] = agent.agent_step(state)
    beta = [] # beta[0] would be first occurence of 0, beta[1] as second occurence and so on...
    #beta.append(0) # generally, this is not counted
    akx = {} # akx is a 2D list which would look like:
    # states -> [1    2]
    # akx    -> [[t1   t2], [t3  t4], [t5    t6],....]
    # For eg. beta = [3, 4, 6, 9] means that 4th recurrence of 0 happens at t=9, when t starts from 0 as usual.
    ikx = {}
    betak = {}
    k = 0
    kt = 0
    rewards = [0 for _ in range(T+1)]
    rewards[0] = 0
    tkx = {}
    wkx = {}
    # following for loop generates an episode
    for t in range(T):
        if state == 0:
            beta.append(t) # gives the successive return epochs to a reference state 0
            # kt = time stamp of betak
            # t1, t2 will be reset with every recurrence of state 0
            kt, t1, t2 = t, math.inf, math.inf  
            k = k + 1
            akx[k] = [t, None, None]
            ikx[k] = [1, None, None]
            betak[k] = t
            tkx[k] = [None, None, None]
            wkx[k] = [None, None, None]
        else:
            if state == 1 and t > kt and t < t1 and kt > 0:
                t1 = t
                akx[k][1] = t
                ikx[k][1] = 1 
            if state == 2 and t > kt and t < t2 and kt > 0:
                t2 = t
                akx[k][2] = t
                ikx[k][2] = 1
        rewards[t+1], state = env.env_step(acs[t])
        sts[t + 1] = state
        acs[t+1] = agent.agent_step(state)

    n = len(beta)
    inx_bar = {0: 0, 1: 0, 2: 0}
    tnx_bar = {0: 0, 1: 0, 2: 0}
    wnx_bar = {0: 0, 1: 0, 2: 0}
    mnx_hat = {0: 0, 1: 0, 2: 0}
    wnx_hat = {0: 0, 1: 0, 2: 0}
    gn_hat = 0
    #hnx_hat = {0: 0, 1: 0, 2: 0} # note that hnx_hat[0] = 0 (mentioned in the paper as well)
    hnx_hat = np.zeros(states)
    for k in range(1, n):
        inx_bar[0] = inx_bar[0] + 1
        wkx[k][0] = sum(rewards[akx[k][0]:beta[k]])
        wnx_bar[0] = wnx_bar[0] + wkx[k][0]
        tkx[k][0] = beta[k] - akx[k][0]
        tnx_bar[0] = tnx_bar[0] + tkx[k][0]

        if ikx[k][1] is not None:
            tkx[k][1] = beta[k] - akx[k][1]
            x = akx[k][1]
            y = beta[k]
            wkx[k][1] = sum(rewards[x:y])
            inx_bar[1] = inx_bar[1] + 1
            wnx_bar[1] = wnx_bar[1] + wkx[k][1]
            tnx_bar[1] = tnx_bar[1] + tkx[k][1]
        if ikx[k][2] is not None:
            tkx[k][2] = beta[k] - akx[k][2]
            x = akx[k][2]
            y = beta[k]
            wkx[k][2] = sum(rewards[x:y])
            inx_bar[2] = inx_bar[2] + 1
            wnx_bar[2] = wnx_bar[2] + wkx[k][2]
            tnx_bar[2] = tnx_bar[2] + tkx[k][2]

    # estimators
    for i in range(3):
        mnx_hat[i] = tnx_bar[i]/inx_bar[i]
        wnx_hat[i] = wnx_bar[i]/inx_bar[i]

    gn_hat = wnx_hat[0]/mnx_hat[0] 
    for i in range(3):
        hnx_hat[i] = wnx_hat[i] - gn_hat*mnx_hat[i]

    dnx = np.zeros(states)
    transition_mat = {0: env.transition_mat_s0, 1: env.transition_mat_s1, 2: env.transition_mat_s2}
    term1 = np.zeros(states)
    term2 = np.zeros(states)
    rx = np.zeros(states)
    for s in range(states):
        rx[s] = np.sum(np.multiply(agent.policy[s].transpose(), env.reward_mat[s]))
        term1[s] = agent.policy[s][0]*np.dot(transition_mat[s][0], hnx_hat)
        term2[s] = agent.policy[s][1]*np.dot(transition_mat[s][1], hnx_hat)
        dnx[s] = gn_hat + hnx_hat[s]*rx[s] - term1[s] - term2[s]

    delta_upper = np.max(dnx)
    delta_lower = np.min(dnx) 
    ung = min(gn_hat, delta_upper)
    lng = delta_lower
    lw_bound = gn_hat - ung
    up_bound = gn_hat - lng

    #Monte Carlo Simulation
    G=0
    for t in range(T - 2, 1, -1):
        G = discount*G + rewards[t+1]
        st = sts[t]
        at = acs[t]
        x = np.argwhere(acs[0:np.max(t-1,0)]==acs[t])
        u= np.argwhere(sts[0:np.max(t-1,0)]==sts[t])
        isRepeat = np.in1d(x,y).any()
        if not isRepeat:
            Returns[st, at, j] = G
            Q[st, at, j] = np.mean(Returns[st, at, :])
            new_action = np.argmax(Q[st, :, j])
            #print(new_action)
            policy[st, new_action] = 1 - epsilon
            policy[st, 1-new_action] = epsilon
        g = Q[0,new_action,j]
        if g > lw_bound and g < up_bound and up_bound - lw_bound < thres:
            flag = 1
            break    

print(policy)
print(g)


[[0.96 0.04]
 [0.96 0.04]
 [0.96 0.04]]
14.527368248612719


In [ ]:
# n = len(beta)
# inx_bar = {0: 0, 1: 0, 2: 0}
# tnx_bar = {0: 0, 1: 0, 2: 0}
# wnx_bar = {0: 0, 1: 0, 2: 0}
# mnx_hat = {0: 0, 1: 0, 2: 0}
# wnx_hat = {0: 0, 1: 0, 2: 0}
# gn_hat = 0
# #hnx_hat = {0: 0, 1: 0, 2: 0} # note that hnx_hat[0] = 0 (mentioned in the paper as well)
# hnx_hat = np.zeros(states)
# for k in range(1, n):
#     inx_bar[0] = inx_bar[0] + 1
#     wkx[k][0] = sum(rewards[akx[k][0]:beta[k]])
#     wnx_bar[0] = wnx_bar[0] + wkx[k][0]
#     tkx[k][0] = beta[k] - akx[k][0]
#     tnx_bar[0] = tnx_bar[0] + tkx[k][0]

#     if ikx[k][1] is not None:
#         tkx[k][1] = beta[k] - akx[k][1]
#         x = akx[k][1]
#         y = beta[k]
#         wkx[k][1] = sum(rewards[x:y])
#         inx_bar[1] = inx_bar[1] + 1
#         wnx_bar[1] = wnx_bar[1] + wkx[k][1]
#         tnx_bar[1] = tnx_bar[1] + tkx[k][1]
#     if ikx[k][2] is not None:
#         tkx[k][2] = beta[k] - akx[k][2]
#         x = akx[k][2]
#         y = beta[k]
#         wkx[k][2] = sum(rewards[x:y])
#         inx_bar[2] = inx_bar[2] + 1
#         wnx_bar[2] = wnx_bar[2] + wkx[k][2]
#         tnx_bar[2] = tnx_bar[2] + tkx[k][2]

# # estimators
# for i in range(3):
#     mnx_hat[i] = tnx_bar[i]/inx_bar[i]
#     wnx_hat[i] = wnx_bar[i]/inx_bar[i]

# gn_hat = wnx_hat[0]/mnx_hat[0] 
# for i in range(3):
#     hnx_hat[i] = wnx_hat[i] - gn_hat*mnx_hat[i]

In [ ]:
# dnx = np.zeros(states)
# transition_mat = {0: env.transition_mat_s0, 1: env.transition_mat_s1, 2: env.transition_mat_s2}
# term1 = np.zeros(states)
# term2 = np.zeros(states)
# rx = np.zeros(states)
# for s in range(states):
#     rx[s] = np.sum(np.multiply(agent.policy[s].transpose(), env.reward_mat[s]))
#     term1[s] = agent.policy[s][0]*np.dot(transition_mat[s][0], hnx_hat)
#     term2[s] = agent.policy[s][1]*np.dot(transition_mat[s][1], hnx_hat)
#     dnx[s] = gn_hat + hnx_hat[s]*rx[s] - term1[s] - term2[s]

# delta_upper = np.max(dnx)
# delta_lower = np.min(dnx) 
# ung = min(gn_hat, delta_upper)
# lng = delta_lower
# lw_bound = gn_hat - ung
# up_bound = gn_hat - lng

In [ ]:
# #Monte Carlo Simulation
# G=0
# for t in range(T - 2, 1, -1):
#     G = discount*G + rewards[t+1]
#     print(G)
#     st = sts[t]
#     at = acs[t]
#     c1 = np.argwhere(acs[0:np.max(t-1,0)]==acs[t])
#     c2 = np.argwhere(sts[0:np.max(t-1,0)]==sts[t])
#     check = np.in1d(c1,c2).any()
#     if not check:
#         Returns[st, at, j] = G
#         Q[st, at, j] = np.mean(Returns[st, at, :])
#         new_action = np.argmax(Q[st, :, j])
#         print(new_action)
#         policy[st, new_action] = 1 - (epsilon/2)
#         policy[st, 1-new_action] = epsilon/2
#         print(policy)
#     g_estimate = Q[0,new_action,j]
#     if g_estimate > lw_bound and g_estimate < up_bound and up_bound - lw_bound < thres:
#         flag = 1;
#         break

# # if flag == 1:
# #     print("flag="+str(flag));
# #     print(j)
# #     break;     

# # print(policy)

    





10.0
1.1
1.011
20.01011
1.2001011
1.012001011
20.01012001011
20.2001012001011
20.202001012001013
1.2020200101200103
20.0120202001012
1.200120202001012
1.0120012020200102
10.0101200120202
1.100101200120202
1.011001012001202
1.010110010120012
1.0101011001012001
1.010101011001012
50.01010101011001
20.5001010101011
20.20500101010101
1.20205001010101
10.012020500101011
1.1001202050010102
1.01100120205001
1.0101100120205
20.010101100120206
1.200101011001202
50.01200101011001
20.5001200101011
50.20500120010101
10.50205001200101
100.10502050012
101.0010502050012
101.01001050205001
21.0101001050205
1.210101001050205
20.0121010100105
50.200121010100105
50.502001210101
50.50502001210101
10.50505020012101
100.1050505020012
101.00105050502002
101.0100105050502
11.010100105050501
1.110101001050505
20.011101010010506
1.2001110101001051
1.0120011101010011
20.01012001110101
50.20010120011101
20.50200101200111
100.20502001012001
51.0020502001012
10.510020502001012
10.10510020502001
10.1010510020502
1.10